In [275]:
import os
import json
import yfinance as yf
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay

PERIOD = 3
LOT = 25000
GAIN_LOSS_CAP = 0.1
HOLDING_PERIOD_CAP = 5  #business days
#Trip = ENTRY,EXIT
#Action = BUY,SELL
#Status = FILLED, PENDING, CANCELED, FORCED
blotter = pd.DataFrame(
    columns=['Date', 'Symbol', 'Trip', 'Action', 'Price', 'Size', 'Status'])

In [276]:
file_path = os.getenv("ITA_DATA_PATH")
data = pd.read_csv(f'{file_path}\data.csv', parse_dates=True, index_col='Date')
data = data.iloc[::-1]
data

,wmt_Open,wmt_High,wmt_Low,wmt_Close,wmt_Volume,amzn_Open,amzn_High,amzn_Low,amzn_Close,amzn_Volume
Date,,,,,,,,,,
2017-01-03,69.24,69.24,68.05,68.66,10473162,757.92,758.76,747.70,753.67,3521066
2017-01-04,68.66,69.63,68.60,69.06,7917952,758.39,759.68,754.20,757.18,2510526
2017-01-05,68.43,69.33,68.12,69.21,7099170,761.55,782.40,760.26,780.45,5830068
2017-01-06,68.41,68.50,68.01,68.26,9491115,782.36,799.44,778.48,795.99,5986234
2017-01-09,68.33,68.80,68.32,68.71,8685232,798.00,801.77,791.77,796.92,3446109
...,...,...,...,...,...,...,...,...,...,...
2022-04-12,154.24,155.40,152.55,153.23,8920400,3073.85,3101.98,3007.66,3015.75,2758872
2022-04-13,153.45,157.79,153.35,157.22,9761532,3000.37,3120.50,2992.00,3110.82,2669544
2022-04-14,157.11,158.29,156.43,157.08,7453493,3107.80,3117.94,3029.44,3034.13,2579907


In [277]:
amzn_hist = data[['amzn_Open', 'amzn_High', 'amzn_Low', 'amzn_Close', 'amzn_Volume']]
wmt_hist = data[['wmt_Open', 'wmt_High', 'wmt_Low', 'wmt_Close', 'wmt_Volume']]

In [278]:
hist = data.copy()
hist['log_ret_AMZN'] = np.log(hist['amzn_Close']) - np.log(hist['amzn_Close'].shift(1))
hist['log_ret_WMT'] = np.log(hist['wmt_Close']) - np.log(hist['wmt_Close'].shift(1))
hist['corr_coef'] = hist['amzn_Close'].rolling(PERIOD).corr(hist['wmt_Close'])
test = 'WMT'
hist

,wmt_Open,wmt_High,wmt_Low,wmt_Close,wmt_Volume,amzn_Open,amzn_High,amzn_Low,amzn_Close,amzn_Volume,log_ret_AMZN,log_ret_WMT,corr_coef
Date,,,,,,,,,,,,,
2017-01-03,69.24,69.24,68.05,68.66,10473162,757.92,758.76,747.70,753.67,3521066,NaN,NaN,NaN
2017-01-04,68.66,69.63,68.60,69.06,7917952,758.39,759.68,754.20,757.18,2510526,0.004646,0.005809,NaN
2017-01-05,68.43,69.33,68.12,69.21,7099170,761.55,782.40,760.26,780.45,5830068,0.030270,0.002170,0.790380
2017-01-06,68.41,68.50,68.01,68.26,9491115,782.36,799.44,778.48,795.99,5986234,0.019716,-0.013821,-0.707053
2017-01-09,68.33,68.80,68.32,68.71,8685232,798.00,801.77,791.77,796.92,3446109,0.001168,0.006571,-0.855904
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-12,154.24,155.40,152.55,153.23,8920400,3073.85,3101.98,3007.66,3015.75,2758872,-0.002216,-0.006894,0.986590
2022-04-13,153.45,157.79,153.35,157.22,9761532,3000.37,3120.50,2992.00,3110.82,2669544,0.031038,0.025706,0.980803
2022-04-14,157.11,158.29,156.43,157.08,7453493,3107.80,3117.94,3029.44,3034.13,2579907,-0.024962,-0.000891,0.672651


# Get Position Direction

In [279]:
def get_position_direction(log_returns):
    amzn_neg_count = np.sum((log_returns['log_ret_AMZN'].values < 0))
    wmt_neg_count = np.sum((log_returns['log_ret_WMT'].values < 0))
    amzn_direction = 'BUY' if (amzn_neg_count >= wmt_neg_count) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'
    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

In [280]:
def get_position_direction2(slice_hist):
    amzn_total_volume = slice_hist['amzn_Volume'].sum()
    amz_negative_volume = np.where(slice_hist['log_ret_AMZN'] < 0, slice_hist['amzn_Volume'], 0).sum()
    amz_negative_pct = amz_negative_volume / amzn_total_volume
    print(amz_negative_pct)

    wmt_total_volume = slice_hist['wmt_Volume'].sum()
    wmt_negative_volume = np.where(slice_hist['log_ret_WMT'] < 0, slice_hist['wmt_Volume'], 0).sum()
    wmt_negative_pct = wmt_negative_volume / wmt_total_volume
    print(wmt_negative_pct)

    amzn_direction = 'BUY' if (amz_negative_pct >= wmt_negative_pct) else 'SELL'
    wmt_direction = 'SELL' if (amzn_direction == 'BUY') else 'BUY'

    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

# Create Entry Trades

In [281]:
def create_entry_trades(biz_date):
    from_date = biz_date - BDay(PERIOD - 1)
    to_date = biz_date
    slice_hist = hist.loc[from_date: to_date, :]

    trade_date = hist[hist.index > biz_date].index[0]  #get next date
    pos_direction = get_position_direction2(slice_hist)
    amzn_open_price = hist.loc[trade_date]['amzn_Open']
    amzn_size = LOT / amzn_open_price
    wmt_open_price = hist.loc[trade_date]['wmt_Open']
    wmt_size = LOT / wmt_open_price

    amzn_entry_trade = {
        "Date": trade_date,
        "Symbol": 'AMZN',
        "Trip": 'ENTRY',
        "Action": pos_direction['AMZN'],
        "Price": amzn_open_price,
        "Size": amzn_size,
        "Status": 'FILLED'
    }
    wmt_entry_trade = {
        "Date": trade_date,
        "Symbol": 'WMT',
        "Trip": 'ENTRY',
        "Action": pos_direction['WMT'],
        "Price": wmt_open_price,
        "Size": wmt_size,
        "Status": 'FILLED'
    }
    return pd.DataFrame([amzn_entry_trade, wmt_entry_trade])

In [282]:
blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'PENDING')].empty

True

# Create Exit Trades

In [283]:
def create_exit_trade(entry_trades_lcl, symbol):
    entry_data = entry_trades_lcl[entry_trades['Symbol'] == symbol].reset_index()
    entry_date = entry_data.at[0, 'Date']
    exit_date = entry_date + BDay(HOLDING_PERIOD_CAP)

    entry_action = entry_data.at[0, 'Action']
    exit_action = 'SELL' if entry_action == 'BUY' else 'BUY'

    entry_price = entry_data.at[0, 'Price']
    exit_price = None
    if exit_action == 'SELL':
        exit_price = entry_price * (1 + GAIN_LOSS_CAP)
    else:
        exit_price = entry_price * (1 - GAIN_LOSS_CAP)

    exit_size = entry_data.at[0, 'Size']

    return {
        "Date": exit_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": exit_action,
        "Price": round(exit_price, 2),
        "Size": exit_size,
        "Status": 'PENDING'
    }

In [284]:
def create_forced_trade(biz_date, symbol, action, size, market_data):
    column_prefix = symbol.lower()
    close_price_column_name = f"{column_prefix}_Close"
    return {
        "Date": biz_date,
        "Symbol": symbol,
        "Trip": 'EXIT',
        "Action": action,
        "Price": market_data[close_price_column_name],
        "Size": size,
        "Status": 'FORCED'
    }

In [285]:
def create_exit_trades(entry_trades_lcl):
    amzn_exit_trade = create_exit_trade(entry_trades_lcl, 'AMZN')
    wmt_exit_trade = create_exit_trade(entry_trades_lcl, 'WMT')
    return pd.DataFrame([amzn_exit_trade, wmt_exit_trade])


In [286]:
def should_force_close(position, biz_date):
    if position['Date'] < biz_date:
        return True


In [287]:
def can_close_position(position, biz_date):
    column_prefix = position['Symbol'].lower()
    high_price_column_name = f"{column_prefix}_High"
    low_price_column_name = f"{column_prefix}_Low"
    if position['Action'] == 'SELL' and hist.loc[biz_date][high_price_column_name] >= position['Price']:
        return True
    if position['Action'] == 'BUY' and hist.loc[biz_date][low_price_column_name] <= position['Price']:
        return True
    return False

In [288]:
for index, today_market_data in hist.iterrows():
    business_date = index
    print(f'*****  Business Date: {business_date}  *****')
    current_pos_status = 'CLOSED' if (blotter.empty or blotter[
        (blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'PENDING')].empty) else 'OPEN'
    # current_pos_status = 'CLOSED'

    if current_pos_status == 'CLOSED' and today_market_data['corr_coef'] < 0:
        entry_trades = create_entry_trades(business_date)
        blotter = pd.concat([blotter, entry_trades], ignore_index=True)

        exit_trades = create_exit_trades(entry_trades)
        blotter = pd.concat([blotter, exit_trades], ignore_index=True)
    elif current_pos_status == 'OPEN':
        pending_trades = blotter[(blotter['Trip'] == 'EXIT') & (blotter['Status'] == 'PENDING')]
        for i, pending_trade in pending_trades.iterrows():
            if should_force_close(pending_trade, business_date):
                print(f"Forcing close on business_date={business_date} for position={pending_trade}")
                blotter.at[i, 'Status'] = 'CANCELED'
                forced_trade = create_forced_trade(business_date,
                                                   pending_trade['Symbol'],
                                                   pending_trade['Action'],
                                                   pending_trade['Size'],
                                                   today_market_data)
                blotter = pd.concat([blotter, pd.DataFrame([forced_trade])], ignore_index=True)
            elif can_close_position(pending_trade, business_date):
                blotter.at[i, 'Status'] = 'FILLED'





*****  Business Date: 2017-01-03 00:00:00  *****
*****  Business Date: 2017-01-04 00:00:00  *****
*****  Business Date: 2017-01-05 00:00:00  *****
*****  Business Date: 2017-01-06 00:00:00  *****
0.0
0.3872622498305366
*****  Business Date: 2017-01-09 00:00:00  *****
*****  Business Date: 2017-01-10 00:00:00  *****
*****  Business Date: 2017-01-11 00:00:00  *****
*****  Business Date: 2017-01-12 00:00:00  *****
*****  Business Date: 2017-01-13 00:00:00  *****
*****  Business Date: 2017-01-17 00:00:00  *****
Forcing close on business_date=2017-01-17 00:00:00 for position=Date      2017-01-16 00:00:00
Symbol                   AMZN
Trip                     EXIT
Action                    BUY
Price                   718.2
Size                31.328321
Status                PENDING
Name: 2, dtype: object
Forcing close on business_date=2017-01-17 00:00:00 for position=Date      2017-01-16 00:00:00
Symbol                    WMT
Trip                     EXIT
Action                   SELL
Price 

In [304]:
print(f"Entry: {blotter[(blotter['Trip']=='ENTRY') & (blotter['Status']=='FILLED')].shape[0]}")
print(f"Exit Filled: {blotter[(blotter['Trip']=='EXIT') & (blotter['Status']=='FILLED')].shape[0]}")
print(f"Exit Forced: {blotter[(blotter['Trip']=='EXIT') & (blotter['Status']=='FORCED')].shape[0]}")

Entry: 264
Exit Filled: 9
Exit Forced: 255
